# Imports and files

In [1]:
import os
from datetime import datetime
from pathlib import Path

import pandas as pd

In [2]:
data_path = Path("data")
analysis_path = Path("benchmark")

stereo_seq_file = data_path / "StereoSeq" / "Mouse_brain_Adult_GEM_bin1.tsv.gz"

signature_file = Path(".") / "yao_brain_signatures_log.tsv"
script_file = Path(".") / "sainsc_benchmark.py"

de_genes_file = data_path / "ABC_atlas" / "DE_genes.xlsx"
ficture_file = (
    Path("FICTURE/minimal_nthread1/analysis/nF42.d_15")
    / "nF42.d_15.posterior.count.tsv.gz"
)

julia_path = Path("julia")
julia_script = julia_path / "benchmark.jl"

In [3]:
analysis_path.mkdir(exist_ok=True, parents=True)

In [4]:
conda_env = "sainsc2"
conda_path = "~/miniconda3/bin/activate"

conda_cmd = f"source {conda_path} {conda_env}"

In [5]:
# ensure submitting to nodes with same hardware
partition = "-p compute-96cpu-700GB-RAM"

# Comparison to FICTURE/TopACT

In [20]:
gene_file = analysis_path / "genes_8k.txt"

pd.read_excel(
    de_genes_file, sheet_name="DE_gene_list", header=None, names=["gene"]
).loc[:, "gene"].to_csv(gene_file, header=False, index=False)

cmd = (
    f"{script_file.resolve()} {stereo_seq_file} {signature_file} "
    "--n_threads 8 "
    f"--genes {gene_file}"
)

id_string = os.popen(
    "sbatch -J sainsc_8k --mem=64G -n 8 -N 1 "
    f"-o {analysis_path/'8k_genes_log.txt'} "
    "--time=3:00:00 "
    "--exclusive "
    f"{partition} "
    f'--wrap="{conda_cmd} && {cmd}" '
).read()

print(id_string)

Submitted batch job 3789462



# Scaling with Threads

Here we are going to use the same genes as FICTURE for comparability.

In [11]:
gene_file = analysis_path / "genes_ficture.txt"

for n_threads in [8, 16, 32, 96]:
    log = analysis_path / f"Threads-{n_threads}.txt"
    cmd = (
        f"{script_file.resolve()} {stereo_seq_file} {signature_file} "
        f"--n_threads {n_threads} "
        f"--genes {gene_file} "
    )

    id_string = os.popen(
        f"sbatch -J sainsc --mem=64G -n {n_threads} -N 1 "
        f"-o {log} "
        "--time=4:00:00 "
        "--exclusive "
        f"{partition} "
        f'--wrap="{conda_cmd} && {cmd}" '
    ).read()

    print(f"Threads: {n_threads}")
    print(id_string)

Threads: 8
Submitted batch job 3788941

Threads: 16
Submitted batch job 3788942

Threads: 32
Submitted batch job 3788943

Threads: 96
Submitted batch job 3788944



### Julia

In [13]:
gene_file = analysis_path / "genes_ficture.txt"


for n_threads in [8, 16, 32, 96]:
    log = analysis_path / f"Threads-{n_threads}_julia.txt"
    cmd = (
        "julia "
        f"--project={julia_path.resolve()} "
        f"--threads={n_threads} "
        "-- "
        f"{julia_script.resolve()} {stereo_seq_file.resolve()} {signature_file.resolve()} {gene_file.resolve()}"
    )
    id_string = os.popen(
        f"sbatch -J sainsc.jl --mem=64G -n {n_threads} -N 1 "
        f"-o {log} "
        "--time=4:00:00 "
        "--exclusive "
        f"{partition} "
        f'--wrap="{cmd}" '
    ).read()

    print(f"Threads: {n_threads}")
    print(id_string)

Threads: 8
Submitted batch job 3788945

Threads: 16
Submitted batch job 3788946

Threads: 32
Submitted batch job 3788947

Threads: 96
Submitted batch job 3788948



# How many genes from the signatures are used?

In [3]:
from sainsc import read_StereoSeq

In [4]:
brain = read_StereoSeq(stereo_seq_file)

In [5]:
signature_genes = pd.read_table(signature_file, index_col=0).index

In [6]:
# 8k ABC atlas DE genes
signature_genes.isin(
    pd.read_excel(
        de_genes_file, sheet_name="DE_gene_list", header=None, names=["gene"]
    )["gene"]
).sum()

7972

In [7]:
# Genes used by Ficture
signature_genes.isin(pd.read_table(ficture_file, usecols=["gene"])["gene"]).sum()

19708

# Scaling with #genes / #signatures

In [21]:
import random

from sainsc import read_StereoSeq

In [22]:
random.seed(42)

In [23]:
signatures = pd.read_table(signature_file, index_col=0)

In [24]:
brain = read_StereoSeq(stereo_seq_file)

In [25]:
signatures = signatures.loc[lambda df: df.index.isin(brain.genes)]

In [16]:
# sample gene sets
for i in range(7):
    n_genes = 250 * 2**i
    genes = random.sample(signatures.index.tolist(), n_genes)
    with open(analysis_path / f"genes_n{n_genes}.txt", "w") as f:
        f.write("\n".join(genes))

In [17]:
# sample signatures
for n_sig in [10, 15, 20, 30, 40]:
    celltypes = random.sample(signatures.columns.tolist(), n_sig)
    signatures.loc[:, celltypes].to_csv(
        analysis_path / f"sig_n{len(celltypes)}.tsv", sep="\t"
    )

In [26]:
jobs = []

for i in range(7):
    for n_sig in [10, 15, 20, 30, 40]:
        n_genes = 250 * 2**i
        gene_file = analysis_path / f"genes_n{n_genes}.txt"
        signature_file = analysis_path / f"sig_n{n_sig}.tsv"

        cmd = (
            f"{script_file.resolve()} {stereo_seq_file} {signature_file} "
            "--n_threads 8 "
            f"--genes {gene_file}"
        )

        id_string = os.popen(
            "sbatch -J sainsc_benchmark --mem=32G -n 8 -N 1 "
            f"-o {analysis_path/f'{n_genes}genes_{n_sig}sigs_log.txt'} "
            "--time=2:00:00 "
            "--exclusive "
            f"{partition} "
            f'--wrap="{conda_cmd} && {cmd}" '
        ).read()

        jobs.append((id_string, n_genes, n_sig))

In [7]:
jobs = []

for n_genes, n_sig in [(1000, 20), (4000, 40), (16000, 10)]:
    gene_file = analysis_path / f"genes_n{n_genes}.txt"
    signature_file = analysis_path / f"sig_n{n_sig}.tsv"

    cmd = (
        f"{script_file.resolve()} {stereo_seq_file} {signature_file} "
        "--n_threads 8 "
        f"--genes {gene_file}"
    )

    id_string = os.popen(
        "sbatch -J sainsc_benchmark --mem=32G -n 8 -N 1 "
        f"-o {analysis_path/f'{n_genes}genes_{n_sig}sigs_log.txt'} "
        "--time=2:00:00 "
        "--exclusive "
        f"{partition} "
        f'--wrap="{conda_cmd} && {cmd}" '
    ).read()

    jobs.append((id_string, n_genes, n_sig))

In [ ]:
jobs = pd.DataFrame(jobs, columns=["jobid", "n_genes", "n_signatures"]).assign(
    jobid=lambda df: df["jobid"].str.extract("(\d+)\s$").astype(int)
)

In [28]:
jobs.to_csv(analysis_path / "sainsc_benchmark_genes_sigs.tsv", sep="\t", index=False)

# Scaling with #transcripts / area

In [15]:
import copy
import random
from math import sqrt

import polars as pl

from sainsc import read_StereoSeq

In [45]:
seed = 42

random.seed(42)

In [17]:
brain = read_StereoSeq(stereo_seq_file)

In [18]:
# make backup of count object to avoid rereading the file
count_bak = copy.copy(brain.counts)

In [19]:
center = tuple(i // 2 for i in brain.shape)

In [42]:
def write_gem(df, path):
    if path.suffix != ".gz":
        path = path.parent / (path.name + ".gz")

    df.rename(columns={"gene": "geneID", "count": "MIDCount"}).to_csv(
        path, sep="\t", index=False
    )

In [46]:
for area in range(20, 101, 20):
    brain.counts = copy.copy(count_bak)
    if area != 100:
        size_factor = sqrt(area / 100)
        radius = tuple(int(i * size_factor / 2) for i in brain.shape)
        crop = tuple((c - r, c + r + 1) for c, r in zip(center, radius))
        brain.counts.crop(*crop)
    transcripts = brain.counts.as_dataframe().to_pandas()
    for subsample in range(20, 101, 20):
        subsampled_tr = transcripts.groupby("gene", observed=True).sample(
            frac=subsample / 100, random_state=seed
        )
        path = analysis_path / f"Brain_area{area}%_subsample{subsample}%.tsv"
        write_gem(subsampled_tr, path)

In [47]:
gene_file = analysis_path / "genes_8k.txt"
signature_file = Path(".") / "yao_brain_signatures_log.tsv"

jobs = []

for area in range(20, 101, 20):
    for subsample in range(20, 101, 20):
        input_file = analysis_path / f"Brain_area{area}%_subsample{subsample}%.tsv.gz"

        cmd = (
            f"{script_file.resolve()} {input_file} {signature_file} "
            "--n_threads 8 "
            f"--genes {gene_file}"
        )

        id_string = os.popen(
            "sbatch -J sainsc_benchmark --mem=32G -n 8 -N 1 "
            f"-o {analysis_path/f'{area}area_{subsample}frac_log.txt'} "
            "--time=2:00:00 "
            "--exclusive "
            f"{partition} "
            f'--wrap="{conda_cmd} && {cmd}" '
        ).read()

        jobs.append((id_string, area, subsample))

In [ ]:
jobs = pd.DataFrame(jobs, columns=["jobid", "area", "fraction_beads"]).assign(
    jobid=lambda df: df["jobid"].str.extract("(\d+)\s$").astype(int)
)

In [49]:
jobs.to_csv(analysis_path / "sainsc_benchmark_area_fraction.tsv", sep="\t", index=False)

# MERFISH

In [6]:
merfish_file = data_path / "MERFISH" / "spots_220613_wb3_sa1_2_5z18R_merfish5.csv"
script_file = Path(".") / "sainsc_benchmark_MERFISH.py"
signature_file = Path(".") / "yao_brain_signatures.tsv"

In [7]:
cmd = f"{script_file.resolve()} {merfish_file} {signature_file} --n_threads 8"

id_string = os.popen(
    "sbatch -J sainsc_merfish --mem=32G -n 8 -N 1 "
    f"-o {analysis_path/'merfish_log.txt'} "
    "--time=1:00:00 "
    "--exclusive "
    f"{partition} "
    f'--wrap="{conda_cmd} && {cmd}" '
).read()

print(id_string)

Submitted batch job 3791196

